<a href="https://colab.research.google.com/github/timomatthies/project-setup/blob/master/HyperLand_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Intitialize


In [38]:
import ee
import geemap
import random
import time
from google.colab import drive
import csv
from IPython.display import Javascript


ee.Authenticate()
ee.Initialize(project='ee-timomatthies17')

### Prepare CSV Files And Counter

In [ ]:
# Initialize export_counter, Get last number from csv file
file_path_bands = '/content/drive/My Drive/EarthEngine_Export/Band-Combinations.csv'

# Open the CSV file in read mode
with open(file_path_bands, mode='r') as file:
    reader = csv.reader(file)
    rows = list(reader)

# Initialize export_counter, Get last number from csv file
file_path = '/content/drive/My Drive/EarthEngine_Export/Metadata.csv'

# Open the CSV file in read mode
with open(file_path, mode='r') as file:
    reader = csv.reader(file)
    rows = list(reader)

# Check if there are rows in the CSV file
if rows:
    # Extract the last row
    last_row = rows[-1]

    # Check if the last row has any items
    if last_row:
        # Read the first item of the last row
        first_item_of_last_row = last_row[0]
        print("First item of the last row:", first_item_of_last_row)
        export_counter = int(first_item_of_last_row) + 1
        print("Current counter:", str(export_counter))
    else:
        print("Last row is empty.")
        export_counter = 1
else:
    export_counter = 1
    print("CSV file is empty.")

First item of the last row: 5
Current counter: 6


### Insert Point and Buffer

In [75]:
#Insert point and buffer here
#point = 1.577, 28.337
#point = 46.277161,23.834345
point = 5.817261, 24.981079
buffer = 45000
buffer_explore = 5000000
buffer_test = 50

# Define a region of interest
roi = ee.Geometry.Point(point[0], point[1])  # Create a buffer around the point

point_lat = point[0]
point_long = point[1]

print("Point Latitude:", point_lat)
print("Point Longitude:", point_long)


Point Latitude: 5.817261
Point Longitude: 24.981079


### Reduce Image Collection

In [80]:

#Load and filter Dataset, apply scaling factors
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

'''
l8_filtered = l8 \
  .filterBounds(roi.buffer(buffer)) \
  .filterDate('2021-05-01', '2021-09-01') \
  .sort("CLOUD_COVER") \

  #.filterMetadata('CLOUD_COVER', 'less_than', 10) \
'''

l8_filtered = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
  .filterDate('2021-05-01', '2021-08-01') \
  .filterMetadata('CLOUD_COVER', 'less_than', 100) \
  .sort("CLOUD_COVER")

buffer_highlight = roi.buffer(buffer)

# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

# Apply scale factors to the dataset
l8_filtered = l8_filtered.map(apply_scale_factors)

# Construct an Image from the Dataset
my_image = l8_filtered.mosaic()

print(type(my_image))
print(type(l8_filtered))

# Generate a random List of available bands
def random_list():
    options = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7", "ST_B10", "ST_ATRAN", "ST_CDIST", "ST_DRAD", "ST_EMIS", "ST_EMSD", "ST_QA"        ]
    return random.sample(options, k=3)

# Example usage:
random_bands = random_list()
print('Random bands:',random_bands)

#selected bands
bands = ['ST_B10', 'SR_B6', 'SR_B4']
#bands = random_bands
print("Current bands:", bands)

# Visualization parameters
visualization = {
    'bands': bands,
     #'bands': random_elements,
    'min': 0.0,
    'max': 1,
    'gamma': 2,
    #'gamma': [0.2,0.2,0.7],
}
# Visualization parameters for the buffer
vis_params_buffer = {
    'color': '0000FF',  # Blue outline color
    'fillColor': '00000000',  # Transparent fill color
    'fillOpacity': 0,  # Full transparency for fill
    'strokeWidth': 2  # Outline width
}

# my_image_vis = my_image.visualize(**viz_params)

Map = geemap.Map(center=(point[1],point[0]), zoom=11, height=800)
# Map.add_basemap(basemap=ROADMAP, show=False)
Map.addLayer(my_image, visualization, 'Mosaic Image')
Map.addLayer(l8_filtered, visualization, 'Filtered Data')
Map.addLayer(buffer_highlight, vis_params_buffer, 'Buffer')
# Map.addLayer(l8_filtered, viz_params, 'Current Test: ' + str(viz_params['bands'][0]))
# Map.add_basemap("SATELLITE")
# Map.add_basemap("OpenTopoMap")
Map

<class 'ee.image.Image'>
<class 'ee.imagecollection.ImageCollection'>
Random bands: ['SR_B4', 'SR_B2', 'SR_B6']
Current bands: ['ST_B10', 'SR_B6', 'SR_B4']


Map(center=[24.981079, 5.817261], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Searc…

#### Interesting Band Combination

In [72]:
print(random_bands)

with open(file_path_bands, mode='a', newline='') as file:

        writer = csv.writer(file)
        # Example data to write row by row
        # rows = [[random_bands[0:3]]] # Print Random Bands
        rows = [[bands[0:3]]] # Print Current Bands
        # Write rows
        print(rows)
        for row in rows:
            writer.writerow(row)

['SR_B6', 'ST_ATRAN', 'SR_B1']
[[['ST_B10', 'SR_B6', 'SR_B4']]]


### Marker Coordinates

In [74]:
marker_coords = Map.draw_last_feature.geometry().getInfo()['coordinates']
marker_coords = str(marker_coords).replace('[', '').replace(']', '')
print("Marker Coordinates:", marker_coords)


Marker Coordinates: 5.817261, 24.981079


### Filename

In [78]:
buffer_used = buffer
# Get image properties
props = geemap.image_props(my_image)

# Get information about the properties
props_info = props.getInfo()
print(props_info)

# Access the system:index property
system_id = props_info.get('system:id')
print(system_id)

result = lambda system_id: system_id if system_id is not None else 'MOSAIC'

filename = (str(export_counter) + "_" + # Nr Export Counter
            'LAT-' + str(point[0]) + # Latitude
            '_LONG-' + str(point[1]) + # Longitude
            '_BUF-' + str(buffer_used) + # Buffer
            '_BANDS-' + str(bands[0:3]).replace('[', '').replace(']', '').replace("'", '').replace(', ', '/')   + # Bands
            '_ID-' +
            result(system_id)) # Image ID

print(filename)

{'NOMINAL_SCALE': 111319.49079327357, 'system:band_names': ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']}
None
7_LAT-5.817261_LONG-24.981079_BUF-45000_BANDS-ST_B10/SR_B6/SR_B4_ID-MOSAIC


### Export

In [79]:
# Export image to Google Drive
export_params = {
    'image': my_image.visualize(**visualization),  # Fix the visualization part
    'fileNamePrefix': filename,
    #'description': 'export_IPTC_test',
    'folder': 'EarthEngine_Export',  # Specify your Google Drive folder
    'region': roi.buffer(buffer_used), #
    'scale': 10,  # Set the scale (resolution) of the exported image
}
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

print('Exporting image to Google Drive...')

# Get the ID of the task after it's started
task_id = task.id
time_past = 0

# Print the task ID
print("Task ID:", task_id)

if task.active():
    print("Export task is currently active.")
else:
    print("Export task is not active.")

while task.active():
    time.sleep(10)  # Wait for 10 seconds before checking again
    time_past = time_past + 10
    print(str(time_past),'s Exporting...')

# Print a message when the export task is done
if task.status()['state'] == 'COMPLETED':

    # Open the CSV file in write mode and specify newline='' to prevent extra newline characters
    with open(file_path, mode='a', newline='') as file:

        result = lambda system_id: system_id if system_id is not None else 'MOSAIC'

        writer = csv.writer(file)

        # Example data to write row by row
        rows = [
            [export_counter,
            point[0],
            point[1],
            buffer_used,
            str(bands[0:3]), #replace('[', '').replace(']', '').replace("'", '').replace(', ', '/'),
            result(system_id),
            ],
        ]
        # Write rows
        for row in rows:
            writer.writerow(row)
    print("CSV was written.")
    export_counter += 1
    print("Export task is done.")

# Check if the task has failed
if task.status()['state'] == 'FAILED':
    error_message = task_status.get('error_message', 'Unknown error')
    print("The task has failed. No CSV Written. Error message:", error_message)

Exporting image to Google Drive...
Task ID: LFGEG355AJ4OKYDIKV4AFFJ3
Export task is currently active.
10 s Exporting...
20 s Exporting...
30 s Exporting...
40 s Exporting...
50 s Exporting...
60 s Exporting...
70 s Exporting...
80 s Exporting...
90 s Exporting...
100 s Exporting...
110 s Exporting...
120 s Exporting...
130 s Exporting...
140 s Exporting...
150 s Exporting...
160 s Exporting...
170 s Exporting...
180 s Exporting...
190 s Exporting...
200 s Exporting...
210 s Exporting...
220 s Exporting...
230 s Exporting...
240 s Exporting...
250 s Exporting...
260 s Exporting...
270 s Exporting...
280 s Exporting...
290 s Exporting...
300 s Exporting...
310 s Exporting...
320 s Exporting...
330 s Exporting...
340 s Exporting...
350 s Exporting...
360 s Exporting...
370 s Exporting...
380 s Exporting...
390 s Exporting...
400 s Exporting...
410 s Exporting...
420 s Exporting...
430 s Exporting...
440 s Exporting...
450 s Exporting...
460 s Exporting...
470 s Exporting...
480 s Exportin

### Check last Task

In [ ]:
# Get a list of all tasks associated with your account
task_list = ee.batch.Task.list()
last_three_tasks = task_list[0:2]

# Iterate through the list of tasks and print their IDs
for task in last_three_tasks:
    print("Task ID:", task.id)
    print("Description:", task.config['description'])
    print("Status:", task.status())
    print("------------------------")

Task ID: JJMKY4CCYJWFUMJTN72BXFCM
Description: export
Status: {'state': 'COMPLETED', 'description': 'export', 'priority': 100, 'creation_timestamp_ms': 1712688832931, 'update_timestamp_ms': 1712688844200, 'start_timestamp_ms': 1712688836784, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1QruD1s8wAhSduLnqKbFL0CVkVwUk1bi6'], 'attempt': 1, 'batch_eecu_usage_seconds': 0.06369349360466003, 'id': 'JJMKY4CCYJWFUMJTN72BXFCM', 'name': 'projects/ee-timomatthies17/operations/JJMKY4CCYJWFUMJTN72BXFCM'}
------------------------
Task ID: ZD2BS4ZTZVIJJY5UFEKPJGYS
Description: export
Status: {'state': 'COMPLETED', 'description': 'export', 'priority': 100, 'creation_timestamp_ms': 1712688698329, 'update_timestamp_ms': 1712688711255, 'start_timestamp_ms': 1712688703062, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1QruD1s8wAhSduLnqKbFL0CVkVwUk1bi6'], 'attempt': 1, 'batch_eecu_usage_seconds': 0.11576125770807266, 'id': 'ZD2BS4

In [ ]:
# ee.batch.Task.cancelAll()

AttributeError: type object 'Task' has no attribute 'cancelAll'